## Workflow Fundamentals

In [3]:
:dep candle-core = "0.8.1"
:dep candle-nn = "0.8.1"
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series"] }

In [4]:
use candle_core::{DType, Device, Module, ModuleT, NdArray, Tensor, Var, D};
use candle_nn::linear::{Linear, linear};
use candle_nn::loss::mse;
use candle_nn::var_builder::{VarBuilder, SimpleBackend};
use candle_nn::{Optimizer, VarMap, SGD};
extern crate plotters;
use plotters::prelude::*;
use std::iter::{zip};


In [7]:
use candle_core::{Device, Tensor, DType};
let device = Device::Cpu;
device.set_seed(42);

### Step 1.  Getting Data Ready

Let's create some data that can be used for linear regression.

One interesting change from the original
```python
X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias
```

Because the tensor operationsl return Result<Tensor>, you have unwrap results of the multiplication before adding the bias.

A way (there are probably better ways), to print the first ten items is to turn the tensor into a vector using once of the methods on the Tensor.

In [8]:
let device = Device::Cpu;
let weight = 0.7;
let bias = 0.3;

let start = 0.0;
let end = 1.0;
let step = 0.02;
let X = Tensor::arange_step(start, end, step, &device)?;
//Do the multiplication first
let y = ((weight * X.clone())? + bias)?;

let x_vec = X.clone().to_vec1::<f64>()?;
let y_vec = y.clone().to_vec1::<f64>()?;
println!("{:?}", &x_vec[..10]);
println!("{:?}", &y_vec[..10]);
let train_split: usize = (0.8 * x_vec.len() as f32) as usize;
println!("Train: {}, Test:{}\n", train_split, x_vec.len() - train_split as usize);

let X_train = Tensor::from_slice(&x_vec[..train_split], (train_split,1), &device)?;
let y_train = Tensor::from_slice(&y_vec[..train_split], (train_split, 1), &device)?;
let x_test = Tensor::from_slice(&x_vec[train_split..x_vec.len()], (x_vec.len() - train_split, 1), &device)?;
let y_test = Tensor::from_slice(&y_vec[train_split..y_vec.len()], (y_vec.len() - train_split, 1), &device)?;


[0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12000000000000001, 0.14, 0.16, 0.18]
[0.3, 0.314, 0.32799999999999996, 0.34199999999999997, 0.356, 0.37, 0.384, 0.398, 0.412, 0.426]
Train: 40, Test:10



### Visualizing the Data

For this we will use plotter-rs.  Candle Tensors can be converted to vecs of different dimensions, so creating data points should be pretty easy

In [ ]:
let training_datapoints: Vec<(f64, f64)> = zip(X_train.squeeze(1)?.to_vec1::<f64>()?, y_train.squeeze(1)?.to_vec1::<f64>()?).collect();
let test_datapoints: Vec<(f64, f64)> = zip(x_test.squeeze(1)?.to_vec1::<f64>()?, y_test.squeeze(1)?.to_vec1::<f64>()?).collect();


In [10]:
evcxr_figure((640, 480), |root| {
    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&root)
        .caption("Training Data", ("Arial", 20).into_font())
        .margin(7)
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f64..1f64, 0f64..1f64)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .draw()?;

    chart.draw_series(training_datapoints
                      .iter()
                      .map(|(x,y)| Circle::new((*x, *y), 2, BLUE.filled())))?
                      .label("Training Data")
                      .legend(|(x,y)| Circle::new((x, y), 2, BLUE.filled()));; 
    
    chart.draw_series(test_datapoints
                  .iter()
                  .map(|(x,y)| Circle::new((*x, *y), 2, GREEN.filled())))?
                  .label("Test Data")
                  .legend(|(x,y)| Circle::new((x, y), 2, GREEN.filled()));; 

    chart.configure_series_labels().position(SeriesLabelPosition::UpperLeft).border_style(BLACK).background_style(WHITE.mix(0.1)).draw()?;
    
    Ok(())
}).style("width: 60%")

Training Data
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Training Data
 
 
Test Data

### 2.  Build Model

Create a linear regression model using candle NN linear layer.

In [11]:
let varmap = VarMap::new();
let vb = VarBuilder::from_varmap(&varmap, DType::F64, &Device::Cpu);

let model: Linear = linear(1, 1, vb)?;

In [ ]:
//Let's look at the initialized weight and bias
println!("Weight: {}", model.weight());
match model.clone().bias() {
    Some(b) => println!("Bias: {}", b),
    None => {}
};


Weight: [[-1.0088]]
Tensor[[1, 1], f64]
Bias: [-0.2959]
Tensor[[1], f64]


Let's have the model make some predications without any training

In [13]:
let y_preds = model.forward(&x_test)?;

In [14]:
let pred_datapoints: Vec<(f64, f64)> = zip(x_test.squeeze(1)?.to_vec1::<f64>()?, y_preds.squeeze(1)?.to_vec1::<f64>()?).collect();

In [ ]:
evcxr_figure((640, 480), |root| {
    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&root)
        .caption("Predication with No Training", ("Arial", 20).into_font())
        .margin(7)
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f64..1f64, -2.5f64..1f64)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .draw()?;

    chart.draw_series(training_datapoints
                      .iter()
                      .map(|(x,y)| Circle::new((*x, *y), 2, BLUE.filled())))?
                      .label("Training Data")
                      .legend(|(x,y)| Circle::new((x, y), 2, BLUE.filled()));; 
    
    chart.draw_series(test_datapoints
                  .iter()
                  .map(|(x,y)| Circle::new((*x, *y), 2, GREEN.filled())))?
                  .label("Test Data")
                  .legend(|(x,y)| Circle::new((x, y), 2, GREEN.filled()));; 

        chart.draw_series(pred_datapoints
                  .iter()
                  .map(|(x,y)| Circle::new((*x, *y), 2, RED.filled())))?
                  .label("Predication")
                  .legend(|(x,y)| Circle::new((x, y), 2, RED.filled()));; 

    chart.configure_series_labels()
        .position(SeriesLabelPosition::UpperLeft)
        .border_style(BLACK)
        .background_style(WHITE.mix(0.1))
        .draw()?;
    
    Ok(())
}).style("width: 60%")

Predication with No Training
 
 
 
-2.5
 
 
 
-2.0
 
 
 
-1.5
 
 
 
-1.0
 
 
 
-0.5
 
 
 
0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Training Data
 
 
Test Data
 
 
Predication

Let's train the model now and see what happens

In [ ]:
let mut train_loss_values: Vec<f64> = vec![];
let mut test_loss_values: Vec<f64> = vec![];
let mut epoch_count: Vec<f64> = vec![];
//Initialize the optimizer with the model weights and biases.
let w = Var::from_tensor(model.weight())?;
let b = Var::from_tensor(model.bias().expect(""))?;
let mut opt = SGD::new(vec![w.clone(), b.clone()], 0.01)?;

for epoch in 0..100 {
    //1.  Forward pass on the training data
    let y_preds = model.forward(&X_train)?;

    //2.  Calculate the loss, for this we will use Mean Squared Error (mse) from candle
    let loss = mse(&y_preds, &y_train)?;
    
    //3.  Candle does not have a zero_grad mechanism
    //4.  Get a GradStore from the loss
    let bp = loss.backward()?;
    //5. progress the optimzer
    opt.step(&bp);

    //Test the model
    let test_pred = model.forward(&x_test)?;
    let test_loss = mse(&test_pred, &y_test)?;
    
    if epoch % 10 == 0 {
        epoch_count.push(epoch as f64);
        train_loss_values.push(loss.to_scalar::<f64>()?);
        test_loss_values.push(test_loss.to_scalar::<f64>()?);
    }
}


()

In [ ]:
// println!("Epochs: {:?}", epoch_count);
// println!("Training Loss Values: {:?}", train_loss_values);
// println!("Test Loss Values: {:?}", test_loss_values);

let train_loss_datapoints: Vec<(f64, f64)> = zip(epoch_count.clone(), train_loss_values.clone()).collect();
let test_loss_datapoints: Vec<(f64, f64)> = zip(epoch_count.clone(), test_loss_values.clone()).collect();
//println!("Train Loss Data Points: {:?}", train_loss_datapoints);
let max_tl: f64 = *test_loss_values.first().unwrap() + 0.05;


In [18]:


evcxr_figure((640, 480), |root| {
    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&root)
        .caption("Training and Test Loss curves", ("Arial", 20).into_font())
        .margin(7)
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f64..100f64, 0f64..max_tl)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .draw()?;

    //chart.draw_series(LineSeries::new(x_values.into_iter().map(|x| (x, 0.3 * x)), BLACK)).unwrap();
    chart.draw_series(LineSeries::new(train_loss_datapoints.clone().into_iter().map(|(x,y)| (x, y)), BLUE))
        .unwrap()
        .label("Training Loss")
        .legend(|(x,y)| Rectangle::new([(x - 10, y + 1), (x, y)], BLUE));
    chart.draw_series(LineSeries::new(test_loss_datapoints.clone().into_iter().map(|(x,y)| (x, y)), RED))
        .unwrap()
        .label("Test Loss")
        .legend(|(x,y)| Rectangle::new([(x - 10, y + 1), (x, y)], RED));

    chart.configure_series_labels()
        .position(SeriesLabelPosition::UpperRight)
        .border_style(BLACK)
        .background_style(WHITE.mix(0.1))
        .draw()?;
    
    Ok(())
}).style("width: 60%")

Training and Test Loss curves
 
 
 
0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
1.5
 
 
 
2.0
 
 
 
2.5
 
 
 
3.0
 
 
 
 
0.0
 
 
 
10.0
 
 
 
20.0
 
 
 
30.0
 
 
 
40.0
 
 
 
50.0
 
 
 
60.0
 
 
 
70.0
 
 
 
80.0
 
 
 
90.0
 
 
 
100.0
 
 
 
 
 
 
 
Training Loss
 
 
Test Loss